# Tiny few shot classifer

Im folgenden wird ein few shot classifier im [SetFit-Frame](https://huggingface.co/blog/setfit) work erzeugt.

## Vorteile von SetFit

Die Verwendung von SetFit (Sentence Transformer Fine-tuning) hat mehrere Vorteile für die hiesige Anwendung:

1. **Few-Shot-Learning**: SetFit wurde speziell entwickelt, um mit sehr wenigen Trainingsbeispielen pro Klasse zu arbeiten (z. B. 8 Beispiele). Das Modell erreicht dennoch eine hohe Genauigkeit, was es ideal für Anwendungsfälle mit begrenzten annotierten Daten macht.

2. **Effiziente Fine-Tuning-Methode**: Anstatt ein großes Sprachmodell vollständig zu trainieren, verwendet SetFit einen zweistufigen Ansatz. Zuerst wird ein vortrainiertes Sentence Transformer-Modell (z. B. MiniLM) auf die Textdaten angepasst, dann wird ein einfacher Klassifikator darauf angewendet. Dies ist sowohl rechen- als auch speichereffizient.

3. **Keine zusätzliche Tokenisierung**: SetFit arbeitet direkt mit Sentence Embeddings, was bedeutet, dass keine zusätzliche Tokenisierung oder Verarbeitung für die Fine-Tuning-Aufgabe erforderlich ist. Dies vereinfacht den Einsatz und spart Rechenressourcen.

4. **Gute Performance bei kleinerem Modell**: Durch die Verwendung von kompakten Sentence Transformers kann SetFit eine hohe Genauigkeit erreichen, ohne die Ressourcenanforderungen eines größeren Modells zu benötigen.

5. **Schnelle Anpassung**: Aufgrund des effizienten Fine-Tuning-Prozesses kann SetFit in kürzerer Zeit auf neue Datensätze angepasst werden, was es für den produktiven Einsatz in schnell wechselnden Umgebungen geeignet macht.

# Berechnung des few shot Modells

## Lade Bibliotheken

Folgende Bibliotheken werden geladen:

- `from datasets import load_dataset`: Zum Laden von Datensätzen aus der HuggingFace `datasets` Bibliothek, z.B. öffentliche oder lokale Datensätze.
  
- `from sentence_transformers.losses import CosineSimilarityLoss`: Wird verwendet, um einen *Cosine Similarity Loss* für das Training von sentence-transformers-Modellen zu definieren (z.B. bei Ähnlichkeitsvergleichen zwischen Textpaaren).
  
- `from setfit import SetFitModel, SetFitTrainer`:  
  - **SetFitModel**: Ein Modell aus dem SetFit-Framework, das für Zero-Shot- oder Few-Shot-Learning verwendet wird.
  - **SetFitTrainer**: Trainer-Klasse, die verwendet wird, um das SetFit-Modell auf spezifische Aufgaben zu trainieren.
  
- `import pandas as pd`: Für die Data-Wrangling
  
- `from sklearn.preprocessing import LabelEncoder`: Zum Kodieren von Zielvariablen (Labels) in numerische Form. Besonders nützlich, wenn die Labels in string-Format vorliegen und in Zahlen umgewandelt werden müssen, um sie einem Modell zu übergeben.
  
- `import os`: Ermöglicht die Interaktion mit dem Betriebssystem, wie z.B. den Zugriff auf Dateipfade, Erstellen von Ordnern oder Lesen von Umgebungsvariablen.
  
- `from datasets import Dataset, DatasetDict`:  
  - **Dataset**: Klasse, um einen benutzerdefinierten Datensatz für die Verwendung mit der `datasets` Bibliothek zu erstellen.
  - **DatasetDict**: Zum Organisieren und Verwalten mehrerer Datensätze (z.B. `train` und `test`).
  
- `from sklearn.model_selection import train_test_split`: Zum Aufteilen eines Datensatzes in Trainings- und Testdaten. Diese Methode wird oft verwendet, um Modelle zu validieren und zu evaluieren.
  
- `from transformers import TrainingArguments`: Legt die Trainingsparameter für Modelle aus der HuggingFace Transformers Bibliothek fest (z.B. Anzahl der Epochen, Batch-Größe, Optimierungsparameter etc.).


In [59]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

from transformers import TrainingArguments

import torch #für gpu-nutzung

##  Daten
### Lade Daten

Um die Daten zu laden wird in einem ersten Schritt das Root-Verzeichnis ausgelesen und ggf. angepasst:

In [52]:
# checke home-verzeichnis
print(os.getcwd())
# setze home-verzeichnis
os.chdir('c:/Users/Hueck/OneDrive/Dokumente/GitHub/future_skill_classification/')

c:\Users\Hueck\OneDrive\Dokumente\GitHub\future_skill_classification


In [ ]:
# Select N examples per class (8 in this case)
import torch
print(torch.cuda.is_available())  # Sollte True ausgeben, wenn CUDA verfügbar ist
print(torch.cuda.get_device_name(0))  # Zeigt den Namen deiner GPU an


True
NVIDIA GeForce RTX 4060 Ti


Stimmt das Verzeichnis, kann im folgenden der Datensatz geladen werden. Wir laden die Daten und schauen uns die ersten

In [53]:
# Laden Ihrer Excel-Datei
df = pd.read_excel('data/fs_test_data.xlsx')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637 entries, 0 to 636
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   titel_kursbeschreibung  637 non-null    object
 1   label_num               637 non-null    int64 
 2   label_text              637 non-null    object
dtypes: int64(1), object(2)
memory usage: 15.1+ KB


### Erzeuge Test- und Trainingsdaten

Im folgenden wird der Datensatz für das Training und die Evaluierung des Clasifiers vorbereitet. 

1. Der Data Frame wird in zwei Teile geteilt: Trainingsdaten (train_df) und Testdaten (test_df). 20% der Daten werden für den Testdatensatz verwendet, während 80% als Trainingsdatensatz verwendet werden. Mit 
`random_state=42` wird sicher gestellt, dass die Aufteilung reproduzierbar ist. 
`stratify=df['label']` sichert schließlich, dass die Verteilung der Zielvariable (label) im Training- und Testdatensatz bleibt, wie im Ursprungsdatensatz `df`.

2. Dataset.from_pandas():
train_df.reset_index(drop=True) und test_df.reset_index(drop=True): Zuerst wird der Index des DataFrames zurückgesetzt, damit keine alte Indexspalte übernommen wird. Das drop=True sorgt dafür, dass der alte Index vollständig entfernt wird.
Dataset.from_pandas(): Diese Methode konvertiert den Pandas DataFrame (train_df und test_df) in ein Dataset-Objekt aus der HuggingFace datasets-Bibliothek, das besser geeignet ist für den Einsatz in NLP-Pipelines.

In [55]:

# Aufteilen des DataFrames
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label_num'])

# Konvertieren in Datasets
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

# Erstellen eines DatasetDicts
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})


## Modell und Trainer initialisieren

Zunächst werden die Klassen `SetFitModel` und `SetFitTrainer` aus der `SetFit`-Bibliothek importiert. `SetFit` basiert auf der sentence-transformers-Architektur und ist speziell für effiziente Few-Shot-Klassifikationsaufgaben konzipiert.

Mit  `paraphrase-mpnet-base-v2` wird ein vortrainiertes Modell geladen, das auf der MPNet-Architektur basiert und darauf spezialisiert ist, semantische Ähnlichkeiten zwischen Texten zu erkennen. Die MPNet-Architektur ist ein Sprachmodell, das darauf trainiert wird, den Kontext und die Bedeutung von Wörtern in einem Satz besser zu verstehen. Es maskiert (versteckt) einige Wörter und versucht, diese anhand des restlichen Satzes vorherzusagen, um so die Bedeutung zu erfassen. Zusätzlich permutiert (vertauscht) es die Reihenfolge der Wörter, um die Wechselwirkungen zwischen ihnen zu lernen. Dadurch kann MPNet komplexe Satzstrukturen und den Sinn von Texten sehr präzise analysieren.

In [56]:
from setfit import SetFitModel, SetFitTrainer
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Als nächstes wird getestet, ob eine GPU (CUDA) vorhanden ist. Ist dies der Fall, wird das Modell mit Hilfe der GPU berechnet. Weiterhin werden folgende Parameter spezifiziert:

- **`SetFitTrainer`:** Dies ist ein Trainer-Objekt, das den gesamten Trainingsprozess steuert, einschließlich der Festlegung der Trainingsdaten, der Verlustfunktion und der Trainingsparameter.

- **`Modell`:** Das Modell, das trainiert wird, ist ein vortrainiertes Sprachmodell, das an die aktuelle Aufgabe angepasst wird.

- **`Trainings- und Testdatensatz`:** `train_dataset` und `eval_dataset` sind die Datensätze, die zum Training und zur Evaluierung des Modells verwendet werden. Der Trainingsdatensatz dient dem Lernen, während der Testdatensatz die Modellleistung prüft.

- **`CosineSimilarityLoss`:** Eine Verlustfunktion, die den Kosinus-Abstand zwischen Satzpaaren misst, um die Ähnlichkeit zwischen ihnen zu bewerten. Sie hilft dem Modell, besser darin zu werden, ähnliche Texte zu erkennen und zu klassifizieren.

- **`batch_size`:** Die Anzahl der Beispiele, die das Modell in einem Durchlauf verarbeitet, bevor es die Modellparameter aktualisiert. Größere Batches können zu stabileren Aktualisierungen führen, benötigen jedoch mehr Speicher.

- **`num_iterations`:** Die Anzahl der Textpaarungen, die pro Beispiel während des Trainings erstellt werden. Mehr Iterationen können zu einer besseren Generalisierung des Modells führen.

- **`num_epochs`:** Die Anzahl der kompletten Durchläufe durch den gesamten Trainingsdatensatz. Mehr Epochen können zu einer verbesserten Modellleistung führen, erhöhen jedoch auch die Trainingszeit.

- **`column_mapping`:** Hier wird festgelegt, welche Spalten des Datensatzes als Texte (`"titel_kursbeschreibung"`) und als Labels (`"label_num"`) verwendet werden sollen. Dies ist wichtig, um dem Modell die Struktur des Datensatzes verständlich zu machen.


In [57]:
# Sicherstellen, dass CUDA verfügbar ist und Gerät festlegen
training_device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


# Modell auf die GPU verschieben
model = model.to(training_device)

# Initialisieren des Trainers mit den Trainingsargumenten
trainer = SetFitTrainer(
    model=model, 
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['test'],
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=10,  # Anzahl der Textpaarungen pro Beispiel
    num_epochs=1,        # Anzahl der Epochen
    column_mapping={"titel_kursbeschreibung": "text", "label_num": "label"}
)



C:\Users\Hueck\AppData\Local\Temp\ipykernel_15616\3209044073.py:13: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset
c:\Users\Hueck\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\utils\_dill.py:379: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


Map:   0%|          | 0/509 [00:00<?, ? examples/s]

Nun wird das Training induziert:

In [58]:
trainer.train()


***** Running training *****
  Num unique pairs = 10180
  Batch size = 16
  Num epochs = 1


  0%|          | 0/637 [00:00<?, ?it/s]

{'embedding_loss': 0.2519, 'grad_norm': 2.2817471027374268, 'learning_rate': 3.125e-07, 'epoch': 0.0}
{'embedding_loss': 0.2065, 'grad_norm': 1.211120843887329, 'learning_rate': 1.5625e-05, 'epoch': 0.08}
{'embedding_loss': 0.1206, 'grad_norm': 2.5888679027557373, 'learning_rate': 1.87434554973822e-05, 'epoch': 0.16}
{'embedding_loss': 0.062, 'grad_norm': 1.6714777946472168, 'learning_rate': 1.699825479930192e-05, 'epoch': 0.24}
{'embedding_loss': 0.0311, 'grad_norm': 0.9450912475585938, 'learning_rate': 1.5253054101221642e-05, 'epoch': 0.31}
{'embedding_loss': 0.0124, 'grad_norm': 1.5740643739700317, 'learning_rate': 1.3507853403141362e-05, 'epoch': 0.39}
{'embedding_loss': 0.0085, 'grad_norm': 0.0821307972073555, 'learning_rate': 1.1762652705061085e-05, 'epoch': 0.47}
{'embedding_loss': 0.0022, 'grad_norm': 0.053271178156137466, 'learning_rate': 1.0017452006980804e-05, 'epoch': 0.55}
{'embedding_loss': 0.0043, 'grad_norm': 0.10809438675642014, 'learning_rate': 8.272251308900523e-06, 

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

{'embedding_loss': 0.0009, 'grad_norm': 0.3505357801914215, 'learning_rate': 3.036649214659686e-06, 'epoch': 0.86}
{'embedding_loss': 0.0012, 'grad_norm': 0.030179228633642197, 'learning_rate': 1.2914485165794066e-06, 'epoch': 0.94}


Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

{'train_runtime': 14351.2202, 'train_samples_per_second': 0.709, 'train_steps_per_second': 0.044, 'train_loss': 0.03563860492134581, 'epoch': 1.0}


## Evaluieren des Modells

In [60]:
import datasets
metrics = trainer.evaluate()

metrics

***** Running evaluation *****


{'accuracy': 0.8828125}

# Upload des Modells

In [62]:
trainer.push_to_hub("Chernoffface/fs-setfit-model")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/37.7k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Chernoffface/fs-setfit-model/commit/d3ad3bd17a0cd533895fae3c22adc590d1deeea1', commit_message='Add SetFit model', commit_description='', oid='d3ad3bd17a0cd533895fae3c22adc590d1deeea1', pr_url=None, pr_revision=None, pr_num=None)